In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, String, Table, MetaData

In [12]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

all_data = []
total_sentimen=dict()
total_pos = 0
total_net = 0
total_neg = 0

model_path = 'model/model_nb.pickle'
modelNB = pickle.load(open(model_path, "rb"))

#Load Vectorizer
vectorizer = TfidfVectorizer()
with open('model/vec.pickle', 'rb') as handle:
    vectorizer = pickle.load(handle)

engine = create_engine('mysql+pymysql://root:@localhost:3306/capres')
conn1 = engine.connect()
Session = sessionmaker(bind=engine)
session = Session()

# #Using pymysql

table_name = 'hasil_preprocessing'  # Replace with the actual table name
specific_word = 'Anies'  # Replace with the specific word you want to search

metadata = MetaData()
metadata.reflect(bind=engine)
table = metadata.tables[table_name]

# conn2 = pymysql.connect(host='localhost', port=int(3306), user='root', password='', db='capres')

query = table.select().where(table.c.cleaning.like(f'%{specific_word}%')).limit(100)
results = session.execute(query)


for result in results:
    
    vec = vectorizer.transform([result.cleaning])
    prediksi = modelNB.predict(vec)
    
    if prediksi == 1:
            hasil_pred = 'POSITIF'
            total_pos += 1
    elif prediksi == 0:
        hasil_pred = 'NETRAL'
        total_net += 1
    else:
        hasil_pred = 'NEGATIF'
        total_neg += 1
    
    all_data.append({
        "Username": result.username,
        "Hasil": result.cleaning,
        "Sentimen": hasil_pred,        
    })
    
total_sentimen["Total_Pos"]= total_pos
total_sentimen["Total_Net"]= total_net
total_sentimen["Total_Neg"]= total_neg
    


f:\FLASK_APP\venv\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
f:\FLASK_APP\venv\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
f:\FLASK_APP\venv\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid re

TypeError: unhashable type: 'slice'

In [9]:
print(all_data["Total_Pos"])

TypeError: list indices must be integers or slices, not str

In [16]:
for item in results:
    print(item.cleaning)